In [81]:
import pandas as pd
train = pd.read_csv("train.tsv", sep="\t", index_col=0) # 学習用データ
test = pd.read_csv("test.tsv", sep="\t", index_col=0)   # 評価用データ
sample_submit = pd.read_csv("sample_submit.tsv", sep="\t", index_col=0, header=None) # 応募用サンプルファイル

In [82]:
train.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,,
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
7,0,3,male,2.0,3,1,21.0750,S
9,1,2,female,14.0,1,0,30.0708,C
11,1,1,female,58.0,0,0,26.5500,S


In [83]:
test.head()

,pclass,sex,age,sibsp,parch,fare,embarked
id,,,,,,,
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
5,3,male,NaN,0,0,8.4583,Q
6,1,male,54.0,0,0,51.8625,S


In [84]:
print(train.shape)
print(test.shape)

(445, 8)
(446, 7)


In [85]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445 entries, 3 to 888
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  445 non-null    int64  
 1   pclass    445 non-null    int64  
 2   sex       445 non-null    object 
 3   age       360 non-null    float64
 4   sibsp     445 non-null    int64  
 5   parch     445 non-null    int64  
 6   fare      445 non-null    float64
 7   embarked  443 non-null    object 
dtypes: float64(2), int64(4), object(2)
memory usage: 31.3+ KB


In [86]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 446 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    446 non-null    int64  
 1   sex       446 non-null    object 
 2   age       354 non-null    float64
 3   sibsp     446 non-null    int64  
 4   parch     446 non-null    int64  
 5   fare      446 non-null    float64
 6   embarked  446 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 27.9+ KB


In [87]:
train['survived'].value_counts()

0    266
1    179
Name: survived, dtype: int64

In [88]:
train['pclass'].value_counts()

3    240
1    108
2     97
Name: pclass, dtype: int64

In [89]:
train['sex'].value_counts()

male      289
female    156
Name: sex, dtype: int64

In [90]:
train['embarked'].value_counts()

S    325
C     79
Q     39
Name: embarked, dtype: int64

In [91]:
train.describe()

,survived,pclass,age,sibsp,parch,fare
count,445.000000,445.000000,360.000000,445.000000,445.000000,445.000000
mean,0.402247,2.296629,29.211583,0.546067,0.431461,33.959971
std,0.490903,0.834024,14.154300,1.195247,0.850489,52.079492
min,0.000000,1.000000,0.670000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.000000,0.000000,0.000000,7.925000
50%,0.000000,3.000000,28.000000,0.000000,0.000000,15.000000
75%,1.000000,3.000000,37.250000,1.000000,1.000000,31.387500
max,1.000000,3.000000,80.000000,8.000000,5.000000,512.329200


In [92]:
pd.get_dummies(train).corrwith(train["survived"])#カテゴリ変数をダミーにしsurvivedとの相関係数

survived      1.000000
pclass       -0.358097
age          -0.081394
sibsp        -0.045087
parch         0.079669
fare          0.258605
sex_female    0.559465
sex_male     -0.559465
embarked_C    0.182568
embarked_Q    0.005062
embarked_S   -0.172769
dtype: float64

In [93]:
train[['pclass', 'survived']].groupby(['pclass']).mean()

,survived
pclass,
1,0.685185
2,0.443299
3,0.258333


In [94]:
train[['sex', 'survived']].groupby(['sex']).mean()

,survived
sex,
female,0.775641
male,0.200692


In [95]:
data = pd.concat([train, test])

In [96]:
data['age'] = data['age'].fillna(data['age'].mean()) 
data['embarked'] = data['embarked'].fillna(data['embarked'].mode()) 

In [97]:
data = pd.get_dummies(data)

In [98]:
data.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
id,,,,,,,,,,,
3,1.0,1,35.0,1,0,53.1000,1,0,0,0,1
4,0.0,3,35.0,0,0,8.0500,0,1,0,0,1
7,0.0,3,2.0,3,1,21.0750,0,1,0,0,1
9,1.0,2,14.0,1,0,30.0708,1,0,1,0,0
11,1.0,1,58.0,0,0,26.5500,1,0,0,0,1


In [99]:
train = data.loc[train.index]
test = data.loc[test.index]
test = test.drop(["survived"], axis=1)

In [100]:
train.head()

,survived,pclass,age,sibsp,parch,fare,sex_female,sex_male,embarked_C,embarked_Q,embarked_S
id,,,,,,,,,,,
3,1.0,1,35.0,1,0,53.1000,1,0,0,0,1
4,0.0,3,35.0,0,0,8.0500,0,1,0,0,1
7,0.0,3,2.0,3,1,21.0750,0,1,0,0,1
9,1.0,2,14.0,1,0,30.0708,1,0,1,0,0
11,1.0,1,58.0,0,0,26.5500,1,0,0,0,1


In [101]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 445 entries, 3 to 888
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   survived    445 non-null    float64
 1   pclass      445 non-null    int64  
 2   age         445 non-null    float64
 3   sibsp       445 non-null    int64  
 4   parch       445 non-null    int64  
 5   fare        445 non-null    float64
 6   sex_female  445 non-null    uint8  
 7   sex_male    445 non-null    uint8  
 8   embarked_C  445 non-null    uint8  
 9   embarked_Q  445 non-null    uint8  
 10  embarked_S  445 non-null    uint8  
dtypes: float64(3), int64(3), uint8(5)
memory usage: 26.5 KB


In [102]:
#モデリング

In [103]:
y = train["survived"] # 目的変数
X = train.drop(["survived"], axis=1) # 目的変数を除いたデータ

In [110]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X, y)

C:\Users\ryooo\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [108]:
pred = model.predict_proba(test)[:, 1] 

In [109]:
sample_submit[1] = pred
sample_submit.to_csv('submit2.tsv', header=None, sep='\t')